In [ ]:
import os
import random

import torch
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F

from AADB.AADB import AADB
from tqdm import tqdm
from utils.utils import *
from models.ResNet18 import RegressionNetwork
from loss_functions.rank_loss import RegRankLoss

In [ ]:
batch_size = 32
epochs = 200
load = False
ckpt = 'F:\Projects\Disertatie\ImageAestheticsGANs\classifier_ckpt_2_pretrained\AADB_epoch_100_loss_0.0702_.pt'

In [ ]:
image_dir = 'F:\Projects\Disertatie\ImageAestheticsGANs\AADB\ImageAesthetics_ECCV2016\datasetImages_warp256'
label_csv_path = 'F:\Projects\Disertatie\ImageAestheticsGANs\AADB\\'

In [ ]:
aadb = AADB(image_dir, label_csv_path, drop_score=False)
aadb_test = AADB(image_dir, label_csv_path, test=True, drop_score=False)

In [ ]:
show_example(*aadb[0])
len(aadb)

In [ ]:
show_example(*aadb_test[0])
len(aadb_test)

In [ ]:
val_size = 500
train_size = len(aadb) - val_size

train_ds, val_ds = random_split(aadb, [train_size, val_size])
len(train_ds), len(val_ds)

In [ ]:
train_dl = DataLoader(train_ds, batch_size*2, shuffle=True, num_workers=0, pin_memory=True, drop_last=True)
valid_dl = DataLoader(val_ds, batch_size, num_workers=0, pin_memory=True)

In [ ]:
print(len(train_dl))
print(len(valid_dl))

In [ ]:
for image, _ in train_dl:
    print(image.shape)
    break

In [ ]:
show_batch(train_dl)

In [ ]:
def get_default_device():
    '''Pick GPU if available'''
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    '''Move tensors to chosen device'''
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True).to(torch.float32)

In [ ]:
class DeviceDataLoader():
    def __init__(self, dl ,device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

In [ ]:
device = get_default_device()
device

In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
valid_dl = DeviceDataLoader(valid_dl, device)

In [ ]:
print(len(train_dl))
print(len(valid_dl))

In [ ]:
model = RegressionNetwork(backbone='resnet18', num_attributes=12, pretrained=True)
model = model.to('cuda')